In [1]:
# 1. Load standard libraries FIRST
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import sys
import importlib
import inspect
import os

# 2. Verify standard libraries are healthy
print(f"Pandas version: {pd.__version__}")

# --- 1. Path Setup ---
# Try to locate the repository root by searching upward for a 'src' directory (or .git)
def find_repo_root(start_path=None, marker_dirs=('src', '.git')):
    p = os.path.abspath(start_path or os.getcwd())
    while True:
        if any(os.path.isdir(os.path.join(p, m)) for m in marker_dirs):
            return p
        parent = os.path.dirname(p)
        if parent == p:
            return None
        p = parent

repo_root = find_repo_root()
# Fallback to previous hardcoded path working on nuvolos
if repo_root is None:
    repo_root = "/files/pixlball"

if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print(f"Using repo_root: {repo_root}")

import src.data as data
import src.model as model
import src.train as train
import src.config as config
import src.dataset as dataset
import src.evaluate as evaluate

from src.config import DEVICE 


# 4. Force a clean reload of your specific logic
importlib.reload(data)
importlib.reload(train)

# 5. THE SMOKE TEST
print("Signature check:", inspect.signature(data.prepare_nn_dataset))

Pandas version: 2.3.3
Using repo_root: c:\Users\jonas\Desktop\repos\pixlball
Signature check: (events_df, nn_layers_df, target_cols=['nn_target'], id_col='id', context_cols=False, keep_context_ids=False)


In [2]:
data_events = pd.read_parquet(os.path.join(repo_root, "data", "events_data.parquet"), engine="fastparquet")
data_360 = pd.read_parquet(os.path.join(repo_root, "data", "sb360_data.parquet"), engine="fastparquet")

In [3]:
admin_events = [
        'Starting XI', 'Half Start', 'Half End', 'Player On', 'Player Off',
        'Substitution', 'Tactical Shift', 'Referee Ball-Drop', 'Injury Stoppage',
        'Bad Behaviour', 'Shield', 'Goal Keeper'
    ]

cleaned_df = data.drop_events(data_events, rows_to_drop=admin_events)

2462 events.


In [4]:
# -----------------------------
# Example usage
# -----------------------------
columns_to_drop = ['clearance_body_part',
                   'clearance_head',
                   'clearance_left_foot',
                   'clearance_other',
                   'clearance_right_foot',
                   'shot_technique',
                   'substitution_replacement_id',
                   'substitution_replacement',
                   'substitution_outcome',
                   'shot_saved_off_target',
                   'pass_miscommunication',
                   'goalkeeper_shot_saved_off_target',
                   'goalkeeper_punched_out',
                   'shot_first_time',
                   'shot_first_time',
                   'shot_body_part',
                   'related_events',
                   'pass_shot_assist', 
                   'pass_straight', 
                   'pass_switch', 
                   'pass_technique', 
                   'pass_through_ball',
                   'goalkeeper_body_part',
                   'goalkeeper_end_location', 
                   'goalkeeper_outcome', 
                   'goalkeeper_position', 
                   'goalkeeper_technique', 
                   'goalkeeper_type', 
                   'goalkeeper_penalty_saved_to_post', 
                   'goalkeeper_shot_saved_to_post', 
                   'goalkeeper_lost_out', 
                   'goalkeeper_Clear', 
                   'goalkeeper_In Play Safe',
                   'shot_key_pass_id',
                   'shot_one_on_one',
                   'shot_end_location',
                   'shot_type',
                   'pass_angle',
                   'pass_body_part',
                   'pass_type',
                   'pass_length',
                   'pass_outswinging',
                   'pass_inswinging',
                   'pass_cross', 
                   'pass_cut_back', 
                   'pass_deflected', 
                   'pass_goal_assist', 
                   'pass_recipient', 
                   'pass_recipient_id', 
                   'pass_assisted_shot_id', 
                   'pass_no_touch', 
                   'pass_end_location', 
                   'pass_aerial_won',
                   'pass_height',
                   'substitution_outcome_id',
                   'tactics',
                   'block_deflection',
                   'dribble_no_touch',
                   'shot_open_goal', 
                   'shot_saved_to_post',
                   'shot_redirect', 
                   'shot_follows_dribble',
                   'period',
                   'injury_stoppage_in_chanin',
                   'block_save_block',
                   'ball recovery_offensive',


                   ]
cleaned_df = data.drop_columns(cleaned_df, columns_to_drop)

# add lookahead outcome
df_with_targets = data.assign_lookahead_outcomes(cleaned_df, lookahead=6)

df_with_targets = data.add_ball_trajectory_features(df_with_targets)


counts of each outcome nn_target
Keep Possession    70920
Lose Possession    27465
Shot                4764
Name: count, dtype: int64


# Prepare 360 Data

In [5]:
df_with_targets

,50_50,ball_receipt_outcome,ball_recovery_recovery_failure,carry_end_location,clearance_aerial_won,counterpress,dribble_nutmeg,dribble_outcome,duel_outcome,duel_type,...,shot_deflected,bad_behaviour_card,block_offensive,foul_committed_offensive,foul_committed_penalty,foul_won_penalty,ball_recovery_offensive,nn_target,goal_flag,ball_trajectory_vector
0,None,None,NaN,None,NaN,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,Lose Possession,0,"[61.0, 40.1, 61.0, 40.1, 61.0, 40.1, 61.0, 40.1]"
1,None,None,NaN,None,NaN,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,Lose Possession,0,"[46.6, 41.5, 61.0, 40.1, 46.6, 41.5, 46.6, 41.5]"
2,None,None,NaN,"[46.4, 41.6]",NaN,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,Lose Possession,0,"[46.6, 41.5, 46.6, 41.5, 61.0, 40.1, 46.6, 41.5]"
3,None,None,NaN,None,NaN,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,Lose Possession,0,"[72.9, 40.9, 46.6, 41.5, 46.6, 41.5, 61.0, 40.1]"
4,None,None,NaN,None,NaN,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,Lose Possession,0,"[46.4, 41.6, 72.9, 40.9, 46.6, 41.5, 46.6, 41.5]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103144,None,None,NaN,None,NaN,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,Shot,1,"[108.1, 40.1, 107.7, 40.1, 108.1, 40.1, 108.1,..."
103145,None,None,NaN,None,NaN,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,Shot,0,"[107.8, 40.1, 108.1, 40.1, 107.7, 40.1, 108.1,..."
103146,None,None,NaN,None,NaN,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,Shot,0,"[107.9, 40.1, 107.8, 40.1, 108.1, 40.1, 107.7,..."
103147,None,None,NaN,None,NaN,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,Shot,0,"[107.9, 40.1, 107.9, 40.1, 107.8, 40.1, 108.1,..."


In [6]:
df_360 = data.assign_grid_cells(data_360)
nn_final = data.aggregate_nn_layers_vectorized(df_360)

# Finalize NN Df

In [11]:
import src.data as data

def prepare_nn_dataset_fixed(
        events_df,
        nn_layers_df,
        target_cols=['nn_target'],
        id_col='id',
        context_cols=False,
        temporal_context=True, 
        keep_context_ids=False
    ):
    # Base columns to keep
    cols_to_keep = [id_col] + target_cols

    # 1. Add Temporal Context (Ball Trajectory Vector)
    if temporal_context and 'ball_trajectory_vector' in events_df.columns:
        cols_to_keep.append('ball_trajectory_vector')

    # 2. If keeping match & possession IDs
    if keep_context_ids:
        for col in ['match_id', 'possession']:
            if col in events_df.columns:
                cols_to_keep.append(col)
        
    # 3. If keeping static context cols
    if context_cols:
        static_features = ['under_pressure', 'counterpress', 'dribble_nutmeg']
        for feat in static_features:
            if feat in events_df.columns:
                cols_to_keep.append(feat)

    events_trimmed = events_df[cols_to_keep].drop_duplicates(subset=id_col)
    return nn_layers_df.merge(events_trimmed, on=id_col, how='inner')

# Overwrite the function in the loaded 'data' module
#data.prepare_nn_dataset = prepare_nn_dataset_fixed

#print("Function successfully patched! You can now run the next cell.")

In [12]:
nn_dataset = data.prepare_nn_dataset(df_with_targets, nn_final, target_cols=['nn_target', 'goal_flag'], context_cols = True, keep_context_ids = True, temporal_context = True ) # adjust cols depending on model

TypeError: prepare_nn_dataset() got an unexpected keyword argument 'temporal_context'

# Neural Network final Data Prep

In [ ]:
context_cols = [
    'under_pressure', 
    'counterpress', 
    'dribble_nutmeg'
]

# Impute NaN values with 0.0 (float)
# This assumes NaN means the event was NOT under pressure, NOT a counterpress, etc.
nn_dataset[context_cols] = nn_dataset[context_cols].fillna(0.0)


target_map = {"Keep Possession": 0, "Lose Possession": 1, "Shot": 2}

# Apply mapping
nn_dataset['nn_target_int'] = nn_dataset['nn_target'].map(target_map)



# Add context Columns
context_df = pd.DataFrame(nn_dataset['ball_trajectory_vector'].tolist(), index=nn_dataset.index)


# Check
print(nn_dataset.head())


                                     id  \
0  8b621ae4-ea81-415c-af41-9669db9bdd93   
1  4706efbe-767c-45aa-9351-09528a77d135   
2  084b9a88-4efa-4947-b94d-b89face472be   
3  27fa7d4d-d637-4487-98e2-5c078ad600c7   
4  764d437f-f799-4489-a38f-69fbb219a6fa   

                                          ball_layer  \
0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....   
1  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....   
2  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....   
3  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....   
4  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....   

                                     teammates_layer  \
0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....   
1  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....   
2  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....   
3  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....   
4  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....   

                                     opponents_layer        nn_targ

# The Goal Multi Task CNN

In [ ]:
from collections import Counter

# ------------------------------------
# 1. Define input columns & targets
# ------------------------------------
# This assumes nn_dataset is already loaded and processed in previous cells.
layer_columns = ["ball_layer", "teammates_layer", "opponents_layer"]

# Ensure labels are in the correct format
event_targets = nn_dataset['nn_target_int'].values   # 0=keep, 1=lose, 2=shot (int)
# CRITICAL: Goal flags must be float for BCEWithLogitsLoss
goal_flags = nn_dataset['goal_flag'].values.astype(np.float32) 

# ------------------------------------
# 3. Compute class weights and positive weight
# ------------------------------------

# A. Event Weights (Multi-Class) - For CrossEntropyLoss
event_counts = Counter(event_targets)
total_events = len(event_targets)

# Using inverse frequency: total / count
class_weights_event = torch.tensor(
    [total_events / event_counts.get(c, 1) for c in range(len(event_counts))],
    dtype=torch.float32
).to(DEVICE)

# B. Goal Positive Weight (Binary) - For BCEWithLogitsLoss
goal_counts = Counter(goal_flags)

STABLE_GOAL_POS_WEIGHT = 3.0
goal_pos_weight = torch.tensor(STABLE_GOAL_POS_WEIGHT, dtype=torch.float32).to(config.DEVICE)

print(f"Goal Positive Weight (0/1 ratio): {goal_pos_weight.item():.2f}")


Goal Positive Weight (0/1 ratio): 3.00


# Preparing the Context CNN

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

layer_columns = ["ball_layer", "teammates_layer", "opponents_layer"]
VALIDATION_SIZE = 0.20
RANDOM_SEED = 42

# --- 1. Define ALL inputs and targets ---
# Input 1: The Grid Layers (X_features)
X_features = nn_dataset[layer_columns].reset_index(drop=True)

# Input 2: The Contextual 1D Features (X_context)
# CRITICAL: Ensure this DataFrame is aligned with X_features
X_context = context_df.reset_index(drop=True)

# Targets
event_targets = nn_dataset['nn_target_int'].values
goal_flags = nn_dataset['goal_flag'].values.astype(np.float32)



# CRITICAL: Assign the 8 returned arrays/DataFrames to 8 descriptive variables
(
    X_feat_train,      # 1. Grid Layers (Train)
    X_feat_val,        # 2. Grid Layers (Validation)
    X_ctx_train,       # 3. Context Features (Train)
    X_ctx_val,         # 4. Context Features (Validation)
    y_event_train,     # 5. Event Targets (Train)
    y_event_val,       # 6. Event Targets (Validation)
    y_goal_train,      # 7. Goal Targets (Train)
    y_goal_val         # 8. Goal Targets (Validation)
) = train_test_split(
    X_features,        # Input 1
    X_context,         # Input 2 (NEW)
    event_targets,     # Input 3
    goal_flags,        # Input 4
    test_size=VALIDATION_SIZE, 
    random_state=RANDOM_SEED,
    stratify=event_targets # Stratify only on the multi-class target
)

# --- 3. Instantiate the two Contextual Dataset objects ---

# Training Dataset (uses four 'train' splits)
train_dataset_context = dataset.ContextBallVectorPitchDatasetMultiTask(
    nn_layers_df=X_feat_train,          # Grid Layers (Train)
    event_targets=y_event_train,        # Event Targets (Train)
    goal_flags=y_goal_train,            # Goal Targets (Train)
    contextual_features_df=X_ctx_train  # Context Features (Train)
)

# Validation Dataset (uses four 'val' splits)
validation_dataset_context = dataset.ContextBallVectorPitchDatasetMultiTask(
    nn_layers_df=X_feat_val,            # Grid Layers (Validation)
    event_targets=y_event_val,          # Event Targets (Validation)
    goal_flags=y_goal_val,              # Goal Targets (Validation)
    contextual_features_df=X_ctx_val    # Context Features (Validation)
)

print(f"Total training samples: {len(train_dataset_context)}")
print(f"Total validation samples: {len(validation_dataset_context)}")

Total training samples: 72117
Total validation samples: 18030


In [ ]:
# Assuming event_class_weights and goal_pos_weight are defined from previous cells
NUM_CONTEXT_FEATURES = 8 

print("Starting training for Contextual CNN Baseline...")

# Modified the Function in Loss to take correct loss function -> needs to be changed for baseline model again

context_baseline_model = train.train_model_context_threat(
    dataset=train_dataset_context, 
    event_class_weights=class_weights_event, # Use your calculated weights
    goal_pos_weight=goal_pos_weight,         # Use your calculated pos_weight
    num_context_features=NUM_CONTEXT_FEATURES
)

print("\nContextual CNN Training complete.")

Starting training for Contextual CNN Baseline...


Context CNN Epoch 1: 100%|██████████| 2254/2254 [00:53<00:00, 42.49it/s, ev_loss=1.5152, loss=1.5152, sh_loss=0.0000] 


Contextual CNN Training complete.


In [ ]:
# Assuming evaluate_model_context is imported and available

print("\nEvaluating Contextual CNN Model...")

metrics = evaluate.evaluate_model_context_threat(
    model=context_baseline_model, 
    dataset=validation_dataset_context # Evaluate on the contextual dataset
)




Evaluating Contextual CNN Model...

--- Event Outcome Metrics ---
Event Accuracy: 0.45568496949528564
Event Balanced Accuracy: 0.5793660770606618
Event Confusion Matrix:
 [[4510 6951 1027]
 [ 951 3071  635]
 [ 101  149  635]]
              precision    recall  f1-score   support

           0       0.81      0.36      0.50     12488
           1       0.30      0.66      0.41      4657
           2       0.28      0.72      0.40       885

    accuracy                           0.46     18030
   macro avg       0.46      0.58      0.44     18030
weighted avg       0.65      0.46      0.47     18030


--- Goal Prediction (xG) Metrics ---
Goal Accuracy: 0.8723163841807909
Goal Balanced Accuracy: 0.5075542207345591
Goal AUC-ROC Score: 0.6064927323582008
Goal Confusion Matrix:
 [[770   2]
 [111   2]]
              precision    recall  f1-score   support

         0.0       0.87      1.00      0.93       772
         1.0       0.50      0.02      0.03       113

    accuracy               

In [ ]:
import numpy as np
# Assuming metrics contains the result from evaluate_model_context_threat

event_probs = metrics['event_probs']

print("P(Keep) | P(Lose) | P(Shot)")
print("-------------------------------")
print(event_probs[:5])

# You can look at the average predicted probability for the Shot class across all events:
avg_p_shot = np.mean(event_probs[:, 2])
print(f"\nAverage Predicted P(Shot) across all events: {avg_p_shot:.4f}")

P(Keep) | P(Lose) | P(Shot)
-------------------------------
[[0.45181692 0.47847646 0.06970669]
 [0.48869404 0.48432195 0.026984  ]
 [0.36873797 0.36380893 0.26745307]
 [0.47087616 0.46040806 0.06871571]
 [0.30941167 0.23137875 0.4592095 ]]

Average Predicted P(Shot) across all events: 0.1484


In [ ]:
import numpy as np
import pandas as pd
# Assuming metrics contains the result from evaluate_model_context_threat

print("--- Goal Prediction Probabilities (xG) Analysis ---")

goal_probs = metrics['goal_probs']
goal_labels = metrics['goal_labels'] # Actual outcome (0=No Goal, 1=Goal)

print(f"Number of Shots Evaluated: {len(goal_probs)}")

# 1. Total xG vs. Actual Goals
total_predicted_xg = np.sum(goal_probs)
total_true_goals = np.sum(goal_labels)
avg_xg_per_shot = np.mean(goal_probs)

print(f"\nTotal Predicted xG: {total_predicted_xg:.2f}")
print(f"Total True Goals Scored: {total_true_goals:.2f}")
print(f"Average Predicted xG per Shot: {avg_xg_per_shot:.4f}")

# 2. Calibration Check (Optional but helpful)
# Compare the average predicted xG for shots that were goals vs. shots that were misses.

# Create a DataFrame for easy slicing
xg_df = pd.DataFrame({'xg': goal_probs, 'goal': goal_labels})

avg_xg_goal = xg_df[xg_df['goal'] == 1]['xg'].mean()
avg_xg_miss = xg_df[xg_df['goal'] == 0]['xg'].mean()

print("\n-- Calibration Check --")
print(f"Average xG for True Goals (should be high): {avg_xg_goal:.4f}")
print(f"Average xG for Missed Shots (should be low): {avg_xg_miss:.4f}")

--- Goal Prediction Probabilities (xG) Analysis ---
Number of Shots Evaluated: 885

Total Predicted xG: 258.79
Total True Goals Scored: 113.00
Average Predicted xG per Shot: 0.2924

-- Calibration Check --
Average xG for True Goals (should be high): 0.3216
Average xG for Missed Shots (should be low): 0.2882
